In [11]:
# from google.colab import drive

# drive.mount("/content/drive")

In [12]:
# %cd "/content/drive/MyDrive/Act2.1"
# !dir

### LEGO Data Classification

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


base_dir = "./"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=25,
    class_mode="binary",
)

Found 4461 images belonging to 16 classes.


In [15]:
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=25,
    class_mode="binary",
)

Found 1918 images belonging to 16 classes.
